# Step : Initialize Core Modules

In [1]:
import sys,os#Add sys to get cmdline_helper from NRPy top directory; remove this line and next when debugged
sys.path.append('../')
import cmdline_helper as cmd     # NRPy+: Multi-platform Python command-line interface

# Create C code output directory:
Ccodesdir = "IMR"
# Then create an output directory in case it does not exist
cmd.mkdir(Ccodesdir)

# Step : Call Initial Conditions
Note : Our new choice of calibration is to obtain $\Delta t$ from BOB at the end of the dynamics. However, if we opt to set $\Delta t$ as a free parameter, then we can feed a value directly. Since the dynamics terminate before ISCO for a positive $\Delta t$, we set the provisional $\Delta t$ for BOB as negative to ensure the dynamics is integrated to one of the ordinary termination conditions. 
LIGO approximants generally provide times in seconds. Thus, the desired sampling `dt` is scaled by the mass factor as EOB uses time in geomtrized units.

In [2]:
%%writefile $Ccodesdir/v5HM_BOB_Generate_Waveform_Calibration.txt
Deltat_init = Deltat
if Delta_t == 'BOB':
    Deltat_init = -1
dT = dt/M/4.925490947641266978197229498498379006e-6
m1,m2,chi1,chi2,y_init,Omega_0,h_init,rstop,rISCO,af,Mf,h22NR,omega22NR = v5HM_BOB_unoptimized_initial_conditions_calibration(M,q,S1,S2,f,a6,dSO,Deltat_init)
omeg22NR *= -1
if af > 0:
    qnm_cache = qnm.modes_cache(s = -2, l = 2, m = 2, n= 0)
    omega_complex, _, _ = qnm_cache(a = af, interp_only = True)
else:
    qnm_cache = qnm.modes_cache(s = -2, l = 2, m = -2, n= 0)
    omega_complex, _, _ = qnm_cache(a = np.abs(af), interp_only = True)

omega_complex_norm = omega_complex/Mf
omega_qnm = np.real(omega_complex_norm)
tau = -1./(np.imag(omega_complex_norm))

Overwriting IMR/v5HM_BOB_Generate_Waveform_Calibration.txt


# Step : Evaluate Dynamics


In [3]:
%%writefile -a $Ccodesdir/v5HM_BOB_Generate_Waveform_Calibration.txt
dynamics_coarse, dynamics_fine = v5HM_BOB_integrator_calibration(m1,m2,chi1,chi2,y_init,Omega_0,a6,dSO,rstop,h_init)
dynamics = np.vstack(dynamics_coarse,dynamics_fine)

Appending to IMR/v5HM_BOB_Generate_Waveform_Calibration.txt


# Step : Find ISCO crossing

Note : pySEOBNR puts in checks for situations where the dynamics are terminated before the perturber crosses the ISCO of the final black hole. In such a case, we choose the end point of the dynamics as the effective ISCO crossing time. (Calibration Notes: In this case, a BOB $\Delta t$ will take in inputs before ISCO. Would be worthwhile to check how accurate the waveform is in this case or if using the BOB $\Delta t$ allows us to push the integration past ISCO.)

In [4]:
%%writefile -a $Ccodesdir/v5HM_BOB_Generate_Waveform_Calibration.txt
if rISCO < dynamics_fine[-1,1]:
    t_ISCO = dynamics_fine[-1,0]
else:
    dt_isco = 0.001
    N = int((dynamics_fine[-1,0] - dynamics_fine[0,0]) / dt_isco)
    zoom = np.linspace(dynamics_fine[0,0],dynamics_fine[-1,0], N)
    n = len(dynamics_fine)
    intrp_r = spline.cspline(n)
    intrp_r.init(dynamics_fine[:,0], dynamics_fine[:,1])
    r_zoomed_in = intrp_r.eval_e_vector(zoom)
    intrp_omega = spline.cspline(n)
    intrp_omega.init(dynamics_fine[:,0], dynamics_fine[:,6])
    omega_zoomed_in = intrp_omega.eval_e_vector(zoom)
    idx = (np.abs(r_zoomed_in - rISCO)).argmin()
    t_ISCO = zoom(idx)
    omega_orb_ISCO = omega_zoomed_in(idx)

Appending to IMR/v5HM_BOB_Generate_Waveform_Calibration.txt


# Compute t_attach

Note: pySEOBNR has checks for situations where the dynamics are terminated before the peak strain time, in which case the merger-ringdown is attached the end time of the dynamics. While pySEOBNR chooses to set the peak strain time at this attachment time, the BOB-generated NQCs makes no assumptions of the attachment time when generating the right-hand-sides.

In [5]:
%%writefile -a $Ccodesdir/v5HM_BOB_Generate_Waveform_Calibration.txt
Delta_t_attach = Delta_t
if Delta_t == 'BOB':
    Omega_QNM = omega_QNM/2
    Omega_QNM4 = Omega_QNM * Omega_QNM * Omega_QNM * Omega_QNM
    Omega_0 = omega22NR/2
    Omega_04 = Omega_0 * Omega_0 * Omega_0 * Omega_0
    omega_orb_ISCO4 = omega_orb_ISCO * omega_orb_ISCO * omega_orb_ISCO * omega_orb_ISCO
    Deltat_t_attach = tau*bob_atanh( ( 2*(omega_orb_ISCO4)*(Omega_QNM4) - Omega_QNM4*Omega_QNM4 - Omega_04*Omega_04 ) / ( Omega_QNM4*Omega_QNM4 - Omega_04*Omega_04 ) ) - 2*tau*np.log(Omega_0/Omega_QNM)

t_peak_strain = t_ISCO - Deltat_attach
t_attach = t_peak_strain
if t_peak_strain > dynamics_fine[-1,0]:
    t_attach = dynamics_fine[-1,0]

Appending to IMR/v5HM_BOB_Generate_Waveform_Calibration.txt


# Compute the fine dynamics waveform and calculate NQC coefficients

In [6]:
%%writefile -a $Ccodesdir/v5HM_BOB_Generate_Waveform_Calibration.txt
h22_inspiral_plunge_fine = get_waveforms_inspiral(m1,m2,dynamics_fine,chi1,chi2)
h22_inspiral_plunge_coarse = get_waveforms_inspiral(m1,m2,dynamics_coarse,chi1,chi2)
nqc_coefficients = v5HM_BOB_unoptimized_nqc_correction(t_peak_strain,t_attach,h22_inspiral_plunge_fine, dynamics_fine)
h22_inspiral_plunge_combined = np.vstack(h22_inspiral_plunge_coarse,h22_inspiral_plunge_fine)
h22_inspiral_plunge_NQC = v5HM_BOB_apply_nqc_correction(nqc_coefficients, h22_inspiral_plung_combined, dynamics)

Appending to IMR/v5HM_BOB_Generate_Waveform_Calibration.txt


# Step:  Compute desired spacing and interpolate dynamics


In [7]:
%%writefile -a $Ccodesdir/v5HM_BOB_Generate_Waveform_Calibration.txt
t_new = np.arange(dynamics[0,0], dynamics[-1,0], dT)
h22_inspiral_plunge = interpolate_modes_fast(t_new,h22_inspiral_plunge_NQC, dynamics)
h22amp_inspiral_plunge = np.abs(h22_inspiral_plunge[:,1])
h22phase_inspiral_plunge = np.unwrap(np.angle(h22_inspiral_plunge[:,1]))

Appending to IMR/v5HM_BOB_Generate_Waveform_Calibration.txt


# Compute the BOB waveform


In [8]:
%%writefile -a $Ccodesdir/v5HM_BOB_Generate_Waveform_Calibration.txt
idx_match = np.argmin(np.abs(t_new - t_attach))
if t_new[idx_attach] > t_attach:
    idx_match -= 1
t_match = t_new[idx_match]
ringdown_time = int(30*tau)
t_BOB = np.arange(0,ringdown_time,dT) + (t_match + dT)
h22amp_BOB, h22phase_BOB = v5HM_BOB_unoptimized_merger_ringdown(t_BOB,t_peak_strain,h22NR,omega22NR,omega_qnm,tau)

Appending to IMR/v5HM_BOB_Generate_Waveform_Calibration.txt


# Compute phase matched BOB waveform


In [9]:
%%writefile -a $Ccodesdir/v5HM_BOB_Generate_Waveform_Calibration.txt
h22phase_match_BOB = h22phase_BOB[0]
h22phase_match_inspiral_plunge = h22phase_inspiral_plunge[idx_match+1]
h22phase_BOB = h22phase_BOB - h22phase_match_BOB + h22phase_match_inspiral_plunge
h22_complex_BOB = h22amp_BOB*np.exp(1j*h22phase_BOB)
h22_BOB = np.c_[t_BOB,h22_complex_BOB]

Appending to IMR/v5HM_BOB_Generate_Waveform_Calibration.txt


# Compute IMR waveform


In [10]:
%%writefile -a $Ccodesdir/v5HM_BOB_Generate_Waveform_Calibration.txt
h22_IMR = np.vstack(h22_inspiral_plunge,h22_BOB)
h22_IMR[:,0] -= t_peak_strain

Appending to IMR/v5HM_BOB_Generate_Waveform_Calibration.txt


# Write a python function

In [11]:
with open(os.path.join(Ccodesdir,"v5HM_BOB_generate_waveform_calibration.py"), "w") as output:
    output.write("import numpy as np\nfrom pygsl_lite import spline\nfrom Dynamics.v5HM_BOB_intial_conditions_calibration import v5HM_BOB_unoptimized_initial_conditions_calibration\nfrom Dynamics.v5HM_BOB_integrator_calibration import v5HM_BOB_integrator_calibration\nfrom Dynamics.v5HM_unoptimized_auxiliary_functions import interpolate_modes_fast, bob_atanh\nfrom IMR.v5HM_BOB_apply_nqc_correction import v5HM_BOB_apply_nqc_correction\nfrom IMR.v5HM_BOB_compute_IMR_waveform import compute_IMR_waveform\nimport qnm\n")
    output.write("from Radiation.v5HM_BOB_unoptimized_merger_ringdown import v5HM_BOB_unoptimized_merger_ringdown\n")
    output.write("def v5HM_BOB_generate_waveform_calibration(M,q,S1,S2,f,a6,dSO,Delta_t,dt,debug = False):\n")
    for line in list(open(os.path.join(Ccodesdir,"v5HM_BOB_Generate_Waveform_Calibration.txt"),"r")):
        output.write("    %s\n" % line.rstrip())
    output.write("    if not debug:\n        return h22_IMR\n    else:\n        return h22_IMR, h22_BOB, h22_inspiral_plunge_NQC, h22_inspiral_plunge_combined, dynamics")